<a href="https://colab.research.google.com/github/harraz/covid-12-predictor/blob/main/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install tensorflow
!pip install seaborn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

In [ ]:
raw_cdc_conditions_by_state=pd.read_csv("/content/drive/MyDrive/CS510/Project/cdc_agg_table_3_20210527.csv")
raw_cdc_conditions_by_state.tail()

In [ ]:
cdc_conditions_by_state=raw_cdc_conditions_by_state.copy()
cdc_conditions_by_state.info()

In [ ]:
# load each csv file in the training data zip file and append to a raw data frame
from zipfile import ZipFile

training_files= '/content/drive/MyDrive/CS510/Project/csse_covid_19_daily_reports/training_set.zip'
test_files= '/content/drive/MyDrive/CS510/Project/csse_covid_19_daily_reports/test_data.zip'

raw_jhu_training_set = pd.concat(
    [pd.read_csv(ZipFile(training_files).open(i)) for i in ZipFile(training_files).namelist()],
    ignore_index=True
)

raw_jhu_testing_set = pd.concat(
    [pd.read_csv(ZipFile(test_files).open(i)) for i in ZipFile(test_files).namelist()],
    ignore_index=True
)

# import glob
# import os

# cols_needed=['Province_State','Deaths','Active','Confirmed','Recovered']

# path = r'/content/drive/MyDrive/CS510/Project/csse_covid_19_daily_reports/training_set'
# all_files = glob.glob(path + "/*.csv")

# all_files = glob.glob(os.path.join(path, "*.csv"))

# df_from_each_file = (pd.read_csv(f, usecols=cols_needed) for f in all_files)
# raw_jhu_training_set   = pd.concat(df_from_each_file, ignore_index=False)

# path = r'/content/drive/MyDrive/CS510/Project/csse_covid_19_daily_reports/test_data'
# all_files = glob.glob(path + "/*.csv")

# all_files = glob.glob(os.path.join(path, "*.csv"))

# df_from_each_file = (pd.read_csv(f, usecols=cols_needed) for f in all_files)
# raw_jhu_testing_set   = pd.concat(df_from_each_file, ignore_index=False)

# raw_jhu_training_set = raw_jhu_training_set[raw_jhu_training_set['Province_State']=='Pennsylvania']
# raw_jhu_training_set

# raw_jhu_testing_set = raw_jhu_testing_set[raw_jhu_testing_set['Province_State']=='Pennsylvania']

# tmp=raw_jhu_testing_set = raw_jhu_testing_set[raw_jhu_testing_set['Active'] ==0]
# tmp


In [ ]:
raw_jhu_testing_set.head()

In [ ]:
states_to_delete=['American Samoa', 'District of Columbia', 'Guam','Marshall Islands', 'Northern Mariana Island','Puerto Rico']
raw_jhu_testing_set = raw_jhu_testing_set[~raw_jhu_testing_set.Province_State.isin(states_to_delete)]
raw_jhu_training_set = raw_jhu_training_set[~raw_jhu_training_set.Province_State.isin(states_to_delete)]

us_idx=raw_jhu_testing_set[raw_jhu_testing_set['Country_Region'] !='US'].index
raw_jhu_testing_set.drop(us_idx,inplace=True)

us_idx=raw_jhu_training_set[ raw_jhu_training_set['Country_Region'] !='US'].index
raw_jhu_training_set.drop(us_idx,inplace=True)

idx=raw_jhu_training_set.query('Confirmed <= 0').index
idx2=(raw_jhu_training_set.query('Active <= 0').index)
idx.append(idx2)
raw_jhu_training_set.drop(idx,inplace=True)
raw_jhu_training_set

idx=raw_jhu_testing_set.query('Confirmed <= 0', engine='python').index
idx2=(raw_jhu_testing_set.query('Active <= 0').index)
idx.append(idx2)
raw_jhu_testing_set.drop(idx,inplace=True)

raw_jhu_training_set.fillna(0,inplace=True)
raw_jhu_testing_set.fillna(0,inplace=True)


In [ ]:
sns.pairplot(raw_jhu_training_set[['Confirmed','Active', 'Deaths', 'Recovered']], diag_kind='kde')

In [ ]:
# join JHU training data with cdc conditions data using state
cdc_jhu_training = pd.merge(raw_jhu_training_set, cdc_conditions_by_state, left_on='Province_State', right_on='LocationDesc', how="inner")
cdc_jhu_training.drop(['LocationDesc', 'LocationAbbr'], axis=1)
cdc_jhu_training = cdc_jhu_training.get(['Province_State','Deaths','Active','Confirmed','OPD','Asthma','Smoking','Poverty','CVD'])


cdc_jhu_testing = pd.merge(raw_jhu_testing_set, cdc_conditions_by_state, left_on='Province_State', right_on='LocationDesc', how="inner")
cdc_jhu_testing.drop(['LocationDesc', 'LocationAbbr'], axis=1)
cdc_jhu_testing = cdc_jhu_testing.get(['Province_State','Deaths','Active','Confirmed','OPD','Asthma','Smoking','Poverty','CVD'])

raw_jhu_testing_set.dropna(inplace=True)
raw_jhu_training_set.dropna(inplace=True)

cdc_jhu_training
# cdc_jhu_training.to_csv('cdc_jhu_training.csv')


In [ ]:
# one hot encoding
cdc_jhu_training['AL']= (cdc_jhu_training.Province_State == 'Alabama') * 1.0
cdc_jhu_training['AK']= (cdc_jhu_training.Province_State == 'Alaska') * 1.0
cdc_jhu_training['AZ']= (cdc_jhu_training.Province_State == 'Arizona') * 1.0
cdc_jhu_training['AR']= (cdc_jhu_training.Province_State == 'Arkansas') * 1.0
cdc_jhu_training['CA']= (cdc_jhu_training.Province_State == 'California') * 1.0
cdc_jhu_training['CO']= (cdc_jhu_training.Province_State == 'Colorado') * 1.0
cdc_jhu_training['CT']= (cdc_jhu_training.Province_State == 'Connecticut') * 1.0
cdc_jhu_training['DE']= (cdc_jhu_training.Province_State == 'Delaware') * 1.0
cdc_jhu_training['FL']= (cdc_jhu_training.Province_State == 'Florida') * 1.0
cdc_jhu_training['GA']= (cdc_jhu_training.Province_State == 'Georgia') * 1.0
cdc_jhu_training['HI']= (cdc_jhu_training.Province_State == 'Hawaii') * 1.0
cdc_jhu_training['ID']= (cdc_jhu_training.Province_State == 'Idaho') * 1.0
cdc_jhu_training['IL']= (cdc_jhu_training.Province_State == 'Illinois') * 1.0
cdc_jhu_training['IN']= (cdc_jhu_training.Province_State == 'Indiana') * 1.0
cdc_jhu_training['IA']= (cdc_jhu_training.Province_State == 'Iowa') * 1.0
cdc_jhu_training['KS']= (cdc_jhu_training.Province_State == 'Kansas') * 1.0
cdc_jhu_training['KY']= (cdc_jhu_training.Province_State == 'Kentucky') * 1.0
cdc_jhu_training['LA']= (cdc_jhu_training.Province_State == 'Louisiana') * 1.0
cdc_jhu_training['ME']= (cdc_jhu_training.Province_State == 'Maine') * 1.0
cdc_jhu_training['MD']= (cdc_jhu_training.Province_State == 'Maryland') * 1.0
cdc_jhu_training['MA']= (cdc_jhu_training.Province_State == 'Massachusetts') * 1.0
cdc_jhu_training['MI']= (cdc_jhu_training.Province_State == 'Michigan') * 1.0
cdc_jhu_training['MN']= (cdc_jhu_training.Province_State == 'Minnesota') * 1.0
cdc_jhu_training['MS']= (cdc_jhu_training.Province_State == 'Mississippi') * 1.0
cdc_jhu_training['MO']= (cdc_jhu_training.Province_State == 'Missouri') * 1.0
cdc_jhu_training['MT']= (cdc_jhu_training.Province_State == 'Montana') * 1.0
cdc_jhu_training['NE']= (cdc_jhu_training.Province_State == 'Nebraska') * 1.0
cdc_jhu_training['NV']= (cdc_jhu_training.Province_State == 'Nevada') * 1.0
cdc_jhu_training['NH']= (cdc_jhu_training.Province_State == 'New Hampshire') * 1.0
cdc_jhu_training['NJ']= (cdc_jhu_training.Province_State == 'New Jersey') * 1.0
cdc_jhu_training['NM']= (cdc_jhu_training.Province_State == 'New Mexico') * 1.0
cdc_jhu_training['NY']= (cdc_jhu_training.Province_State == 'New York') * 1.0
cdc_jhu_training['NC']= (cdc_jhu_training.Province_State == 'North Carolina') * 1.0
cdc_jhu_training['ND']= (cdc_jhu_training.Province_State == 'North Dakota') * 1.0
cdc_jhu_training['OH']= (cdc_jhu_training.Province_State == 'Ohio') * 1.0
cdc_jhu_training['OK']= (cdc_jhu_training.Province_State == 'Oklahoma') * 1.0
cdc_jhu_training['OR']= (cdc_jhu_training.Province_State == 'Oregon') * 1.0
cdc_jhu_training['PA']= (cdc_jhu_training.Province_State == 'Pennsylvania') * 1.0
cdc_jhu_training['RI']= (cdc_jhu_training.Province_State == 'Rhode Island') * 1.0
cdc_jhu_training['SC']= (cdc_jhu_training.Province_State == 'South Carolina') * 1.0
cdc_jhu_training['SD']= (cdc_jhu_training.Province_State == 'South Dakota') * 1.0
cdc_jhu_training['TN']= (cdc_jhu_training.Province_State == 'Tennessee') * 1.0
cdc_jhu_training['TX']= (cdc_jhu_training.Province_State == 'Texas') * 1.0
cdc_jhu_training['UT']= (cdc_jhu_training.Province_State == 'Utah') * 1.0
cdc_jhu_training['VT']= (cdc_jhu_training.Province_State == 'Vermont') * 1.0
cdc_jhu_training['VA']= (cdc_jhu_training.Province_State == 'Virginia') * 1.0
cdc_jhu_training['WA']= (cdc_jhu_training.Province_State == 'Washington') * 1.0
cdc_jhu_training['WV']= (cdc_jhu_training.Province_State == 'West Virginia') * 1.0
cdc_jhu_training['WI']= (cdc_jhu_training.Province_State == 'Wisconsin') * 1.0
cdc_jhu_training['WY']= (cdc_jhu_training.Province_State == 'Wyoming') * 1.0
# cdc_jhu_training['AS']= (cdc_jhu_training.Province_State == 'American Samoa') * 1.0
# cdc_jhu_training['DC']= (cdc_jhu_training.Province_State == 'District of Columbia') * 1.0
# cdc_jhu_training['GU']= (cdc_jhu_training.Province_State == 'Guam') * 1.0
# cdc_jhu_training['MH']= (cdc_jhu_training.Province_State == 'Marshall Islands') * 1.0
# cdc_jhu_training['MP']= (cdc_jhu_training.Province_State == 'Northern Mariana Island') * 1.0
# cdc_jhu_training['PR']= (cdc_jhu_training.Province_State == 'Puerto Rico') * 1.0

# do the same for test data set
cdc_jhu_testing['AL']= (cdc_jhu_testing.Province_State == 'Alabama') * 1.0
cdc_jhu_testing['AK']= (cdc_jhu_testing.Province_State == 'Alaska') * 1.0
cdc_jhu_testing['AZ']= (cdc_jhu_testing.Province_State == 'Arizona') * 1.0
cdc_jhu_testing['AR']= (cdc_jhu_testing.Province_State == 'Arkansas') * 1.0
cdc_jhu_testing['CA']= (cdc_jhu_testing.Province_State == 'California') * 1.0
cdc_jhu_testing['CO']= (cdc_jhu_testing.Province_State == 'Colorado') * 1.0
cdc_jhu_testing['CT']= (cdc_jhu_testing.Province_State == 'Connecticut') * 1.0
cdc_jhu_testing['DE']= (cdc_jhu_testing.Province_State == 'Delaware') * 1.0
cdc_jhu_testing['FL']= (cdc_jhu_testing.Province_State == 'Florida') * 1.0
cdc_jhu_testing['GA']= (cdc_jhu_testing.Province_State == 'Georgia') * 1.0
cdc_jhu_testing['HI']= (cdc_jhu_testing.Province_State == 'Hawaii') * 1.0
cdc_jhu_testing['ID']= (cdc_jhu_testing.Province_State == 'Idaho') * 1.0
cdc_jhu_testing['IL']= (cdc_jhu_testing.Province_State == 'Illinois') * 1.0
cdc_jhu_testing['IN']= (cdc_jhu_testing.Province_State == 'Indiana') * 1.0
cdc_jhu_testing['IA']= (cdc_jhu_testing.Province_State == 'Iowa') * 1.0
cdc_jhu_testing['KS']= (cdc_jhu_testing.Province_State == 'Kansas') * 1.0
cdc_jhu_testing['KY']= (cdc_jhu_testing.Province_State == 'Kentucky') * 1.0
cdc_jhu_testing['LA']= (cdc_jhu_testing.Province_State == 'Louisiana') * 1.0
cdc_jhu_testing['ME']= (cdc_jhu_testing.Province_State == 'Maine') * 1.0
cdc_jhu_testing['MD']= (cdc_jhu_testing.Province_State == 'Maryland') * 1.0
cdc_jhu_testing['MA']= (cdc_jhu_testing.Province_State == 'Massachusetts') * 1.0
cdc_jhu_testing['MI']= (cdc_jhu_testing.Province_State == 'Michigan') * 1.0
cdc_jhu_testing['MN']= (cdc_jhu_testing.Province_State == 'Minnesota') * 1.0
cdc_jhu_testing['MS']= (cdc_jhu_testing.Province_State == 'Mississippi') * 1.0
cdc_jhu_testing['MO']= (cdc_jhu_testing.Province_State == 'Missouri') * 1.0
cdc_jhu_testing['MT']= (cdc_jhu_testing.Province_State == 'Montana') * 1.0
cdc_jhu_testing['NE']= (cdc_jhu_testing.Province_State == 'Nebraska') * 1.0
cdc_jhu_testing['NV']= (cdc_jhu_testing.Province_State == 'Nevada') * 1.0
cdc_jhu_testing['NH']= (cdc_jhu_testing.Province_State == 'New Hampshire') * 1.0
cdc_jhu_testing['NJ']= (cdc_jhu_testing.Province_State == 'New Jersey') * 1.0
cdc_jhu_testing['NM']= (cdc_jhu_testing.Province_State == 'New Mexico') * 1.0
cdc_jhu_testing['NY']= (cdc_jhu_testing.Province_State == 'New York') * 1.0
cdc_jhu_testing['NC']= (cdc_jhu_testing.Province_State == 'North Carolina') * 1.0
cdc_jhu_testing['ND']= (cdc_jhu_testing.Province_State == 'North Dakota') * 1.0
cdc_jhu_testing['OH']= (cdc_jhu_testing.Province_State == 'Ohio') * 1.0
cdc_jhu_testing['OK']= (cdc_jhu_testing.Province_State == 'Oklahoma') * 1.0
cdc_jhu_testing['OR']= (cdc_jhu_testing.Province_State == 'Oregon') * 1.0
cdc_jhu_testing['PA']= (cdc_jhu_testing.Province_State == 'Pennsylvania') * 1.0
cdc_jhu_testing['RI']= (cdc_jhu_testing.Province_State == 'Rhode Island') * 1.0
cdc_jhu_testing['SC']= (cdc_jhu_testing.Province_State == 'South Carolina') * 1.0
cdc_jhu_testing['SD']= (cdc_jhu_testing.Province_State == 'South Dakota') * 1.0
cdc_jhu_testing['TN']= (cdc_jhu_testing.Province_State == 'Tennessee') * 1.0
cdc_jhu_testing['TX']= (cdc_jhu_testing.Province_State == 'Texas') * 1.0
cdc_jhu_testing['UT']= (cdc_jhu_testing.Province_State == 'Utah') * 1.0
cdc_jhu_testing['VT']= (cdc_jhu_testing.Province_State == 'Vermont') * 1.0
cdc_jhu_testing['VA']= (cdc_jhu_testing.Province_State == 'Virginia') * 1.0
cdc_jhu_testing['WA']= (cdc_jhu_testing.Province_State == 'Washington') * 1.0
cdc_jhu_testing['WV']= (cdc_jhu_testing.Province_State == 'West Virginia') * 1.0
cdc_jhu_testing['WI']= (cdc_jhu_testing.Province_State == 'Wisconsin') * 1.0
cdc_jhu_testing['WY']= (cdc_jhu_testing.Province_State == 'Wyoming') * 1.0
# cdc_jhu_testing['AS']= (cdc_jhu_testing.Province_State == 'American Samoa') * 1.0
# cdc_jhu_testing['DC']= (cdc_jhu_testing.Province_State == 'District of Columbia') * 1.0
# cdc_jhu_testing['GU']= (cdc_jhu_testing.Province_State == 'Guam') * 1.0
# cdc_jhu_testing['MH']= (cdc_jhu_testing.Province_State == 'Marshall Islands') * 1.0
# cdc_jhu_testing['MP']= (cdc_jhu_testing.Province_State == 'Northern Mariana Island') * 1.0
# cdc_jhu_testing['PR']= (cdc_jhu_testing.Province_State == 'Puerto Rico') * 1.0


In [ ]:
cdc_jhu_training

In [ ]:
# drop some state columns 
# cdc_jhu_training.drop(cdc_jhu_training.columns[7:],axis=1,inplace=True)

# cdc_jhu_testing.drop(cdc_jhu_testing.columns[7:],axis=1,inplace=True)

cdc_jhu_training.pop('Province_State')
cdc_jhu_testing.pop('Province_State')
cdc_jhu_training.describe()

In [ ]:
sns.pairplot(cdc_jhu_training[['Deaths','Confirmed','Active','Poverty', 'CVD', 'OPD','Asthma']], diag_kind='kde')

In [ ]:
train_stats = cdc_jhu_training.describe()
train_stats.pop('Deaths')
train_stats = train_stats.transpose()
train_stats

In [ ]:
train_labels = cdc_jhu_training.pop('Deaths')
test_labels = cdc_jhu_testing.pop('Deaths')

train_labels

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(cdc_jhu_training)
normed_test_data = norm(cdc_jhu_testing)

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.sigmoid, input_shape=[len(normed_train_data.keys())]),
    layers.Dense(64, activation=tf.nn.sigmoid),
    layers.Dense(64, activation=tf.nn.sigmoid),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  # optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mse','mae'])
  return model

In [ ]:
model=build_model()

In [ ]:
model.summary()

In [ ]:
example_batch = normed_train_data[:10]
example_batch

example_result = model.predict(example_batch)
example_result


In [ ]:
# display training progress
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)
history = model.fit(
    x=normed_train_data, y=train_labels, batch_size=None, epochs=EPOCHS, verbose='auto',
    callbacks=[early_stop], validation_split=0.2, validation_data=None, shuffle=True,
    class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None,
    validation_steps=None, validation_batch_size=None, validation_freq=1,
    max_queue_size=10, workers=1, use_multiprocessing=True
)

# history = model.fit(
#     normed_train_data, train_labels, 
#     epochs=EPOCHS, validation_split = 0.2, verbose=0,
#     callbacks=[PrintDot()])

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
history.history.keys()

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Absolute Error [Deaths]') 
  plt.plot(hist['epoch'], hist['mae'], label ='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'], label = 'Validation Error')
  plt.gca()
  plt.yscale('log')
  plt.legend()
  plt.ylim([10,3000])

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$Deaths^2$]') 
  plt.plot(hist['epoch'], hist['mse'], label ='Train Error')
  # plt.plot(hist['epoch'], hist['val_mse'], label = 'Validation Mean Squared Error')
  # plt.yscale('log')
  plt.legend()
  plt.ylim([20000,250000])
plot_history(history)


In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)
print("Testing set mean Absolute Error: {:5.7f} Deaths".format(mae))

#Testing set mean Absolute Error: 2393219.0000000 Deaths
#Testing set mean Absolute Error: 1033921.6250000 Deaths (add layer)
#Testing set mean Absolute Error: 1637497.3750000 Deaths (remove layer and use sigmoid)
#Testing set mean Absolute Error: 582547.6250000 Deaths
#Testing set mean Absolute Error: 1449834.2500000 Deaths
#Testing set mean Absolute Error: 1420743.2500000 Deaths (added layer and patience to 20)
# Testing set mean Absolute Error: 991202.8750000 Deaths (changed optimizer to )


In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel("True Values [Deaths]")
plt.ylabel("Predictions [Deaths]")
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-10,10],[-10,10])

In [ ]:
err = test_predictions - test_labels
plt.hist(err, bins=10)
plt.xlabel("Predictin Error [Deaths]")
_ = plt.ylabel("count")